# Imports

In [1]:
import pandas as pd
import numpy as np

from fetching_ratings import get_user_ratings
from app_functions import unzip_and_load_datasets, normalise_scores

## Loading Data

### My Ratings

In [2]:
df_my_ratings = get_user_ratings()
df_my_ratings

Retrieving ratings from page 1/3
Content retrieved: 100

Retrieving ratings from page 2/3
Content retrieved: 200

Retrieving ratings from page 3/3
Content retrieved: 261



,tconst,userRating
0,tt16968450,4
1,tt1517268,5
2,tt10478048,7
3,tt14230458,7
4,tt17351924,6
...,...,...
256,tt7366338,8
257,tt2707408,9
258,tt0111161,10
259,tt0944947,9


### IMDB Data

In [3]:
datasets = unzip_and_load_datasets()

In [23]:
datasets[1]

,tconst,averageRating,numVotes
0,tt0000001,5.7,2024
1,tt0000002,5.7,272
2,tt0000003,6.5,1957
3,tt0000004,5.4,178
4,tt0000005,6.2,2724
...,...,...,...
1402534,tt9916730,7.6,11
1402535,tt9916766,7.1,23
1402536,tt9916778,7.2,36
1402537,tt9916840,8.8,6


In [37]:
# Get series and episode data
df_imdb_titles = datasets[0]
df_imdb_titles = df_imdb_titles[df_imdb_titles['titleType'].isin(['tvSeries', 'tvMiniSeries', 'tvEpisode'])]
num_series = len(df_imdb_titles) - (df_imdb_titles['titleType'] == 'tvEpisode').sum()
print('# SERIES:', num_series)

# Get ratings for series and episodes
df_imdb_ratings = datasets[1]
df_imdb_titles = df_imdb_titles.merge(df_imdb_ratings, on='tconst')
num_rated_series = len(df_imdb_titles) - (df_imdb_titles['titleType'] == 'tvEpisode').sum()
print('# SERIES WITH RATINGS: {} ({}%)'.format(num_rated_series, round(100*num_rated_series / num_series)))

# Get episode info
df_imdb_episodes = datasets[2]
df_imdb_episodes = df_imdb_episodes.merge(
    df_imdb_titles.loc[df_imdb_titles['titleType'] == 'tvEpisode', ['tconst', 'runtimeMinutes', 'averageRating', 'numVotes']],
    on='tconst'
)
# df_imdb_titles = df_imdb_titles.merge(df_imdb_episodes, on='tconst', how='left')

df_imdb_titles['titleType'].value_counts()

# SERIES: 309504
# SERIES WITH RATINGS: 109900 (36%)


tvEpisode       696889
tvSeries         93255
tvMiniSeries     16645
Name: titleType, dtype: int64

# Steps
- Rank series by score (combination of number of votes and series ratings)
- Rank series by average episode score (combination of number of votes and episode ratings)
- Rank series by total runtime (sum of episode runtimes)

In [38]:
df_imdb_titles.loc[df_imdb_titles['titleType'].isin(['tvSeries', 'tvMiniSeries'])]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0035803,tvSeries,The German Weekly Review,Die Deutsche Wochenschau,0,1940,1945,\N,"Documentary,News",8.0,60
1,tt0039120,tvSeries,Americana,Americana,0,1947,1949,30,"Family,Game-Show",2.7,18
2,tt0039123,tvSeries,Kraft Theatre,Kraft Television Theatre,0,1947,1958,60,Drama,8.0,217
3,tt0039125,tvSeries,Public Prosecutor,Public Prosecutor,0,1947,1951,20,"Crime,Drama,Mystery",5.7,29
4,tt0040021,tvSeries,Actor's Studio,Actor's Studio,0,1948,1950,30,Drama,6.9,88
...,...,...,...,...,...,...,...,...,...,...,...
806745,tt9915822,tvSeries,Ichhapyaari Naagin,Ichhapyaari Naagin,0,2016,2020,20,Fantasy,7.9,66
806757,tt9916128,tvSeries,Salt City (India),Salt City (India),0,2022,\N,\N,Drama,6.1,120
806760,tt9916206,tvSeries,Nojor,Nojor,0,2019,\N,20,Fantasy,3.2,33
806761,tt9916216,tvSeries,Kalyanam Mudhal Kadhal Varai,Kalyanam Mudhal Kadhal Varai,0,2014,2017,22,Romance,8.7,18


In [39]:
df_imdb_episodes

,tconst,parentTconst,seasonNumber,episodeNumber,runtimeMinutes,averageRating,numVotes
0,tt0041951,tt0041038,1,9,30,7.6,87
1,tt0042816,tt0989125,1,17,135,7.6,12
2,tt0044093,tt0959862,1,6,60,4.5,18
3,tt0045960,tt0044284,2,3,75,6.9,190
4,tt0046150,tt0341798,\N,\N,84,8.2,12
...,...,...,...,...,...,...,...
696880,tt9916708,tt0985991,3,48,\N,8.6,6
696881,tt9916766,tt1442550,10,15,43,7.1,23
696882,tt9916778,tt9595170,1,3,\N,7.2,36
696883,tt9916840,tt0985991,4,1,11,8.8,6


## Score


In [50]:
df_series_score = df_imdb_titles.loc[df_imdb_titles['titleType'].isin(['tvSeries', 'tvMiniSeries'])].copy()
df_series_score['seriesScore'] = df_series_score['averageRating'] * df_series_score['numVotes']
df_series_score = normalise_scores(df_series_score, score_col='seriesScore')
df_series_score['seriesRank'] = df_series_score['seriesScore'].rank(method='min', ascending=False).astype(int) # Ranks 1,2,2,4,5,6,6,8 style
df_series_score.sort_values('seriesRank', inplace=True)
df_series_score.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,seriesScore,seriesRank
183789,tt0903747,tvSeries,Breaking Bad,Breaking Bad,0,2008,2013,45,"Crime,Drama,Thriller",9.5,2100308,9.75,1
192049,tt0944947,tvSeries,Game of Thrones,Game of Thrones,0,2011,2019,4189,"Action,Adventure,Drama",9.2,2254188,9.60,2
737266,tt7366338,tvMiniSeries,Chernobyl,Chernobyl,0,2019,2019,330,"Drama,History,Thriller",9.3,848340,9.54,3
368299,tt1475582,tvSeries,Sherlock,Sherlock,0,2010,2017,88,"Crime,Drama,Mystery",9.1,986352,9.48,4
6506,tt0108778,tvSeries,Friends,Friends,0,1994,2004,5280,"Comedy,Romance",8.9,1074934,9.40,5


## Episode Quality

In [41]:
df_imdb_episodes['episodeScore'] = df_imdb_episodes['averageRating'] * df_imdb_episodes['numVotes']
df_imdb_episodes = normalise_scores(df_imdb_episodes, score_col='episodeScore')
df_imdb_episodes

,tconst,parentTconst,seasonNumber,episodeNumber,runtimeMinutes,averageRating,numVotes,episodeScore
433186,tt2301451,tt0903747,5,14,47,10.0,211875,10.00
538652,tt4283088,tt0944947,6,9,60,9.9,222405,9.95
538654,tt4283094,tt0944947,6,10,68,9.9,158453,9.95
433188,tt2301455,tt0903747,5,16,55,9.9,137370,9.94
418967,tt2178784,tt0944947,3,9,51,9.9,116245,9.92
...,...,...,...,...,...,...,...,...
535072,tt4175520,tt3893314,1,1,\N,1.0,5,3.00
235819,tt11880296,tt11229450,1,26,\N,1.0,5,3.00
552212,tt4703550,tt4680034,1,4,\N,1.0,5,3.00
535081,tt4175544,tt3893314,1,4,45,1.0,5,3.00


In [64]:
# Group episodes by the series to which they belong and calculate their mean score
df_episode_score = df_imdb_episodes.groupby('parentTconst').agg({'episodeScore': 'mean'})
df_episode_score.reset_index(inplace=True)

df_episode_score['episodeRank'] = df_episode_score['episodeScore'].rank(method='min', ascending=False).astype(int)
df_episode_score.sort_values('episodeRank', inplace=True)
df_episode_score.head()

,parentTconst,episodeScore,episodeRank
33250,tt7366338,9.284000,1
27486,tt3581920,9.063333,2
11684,tt11198330,8.706000,3
9556,tt0944947,8.638356,4
9382,tt0903747,8.355000,5


## Episode Runtime

In [47]:
df_imdb_episodes['runtimeMinutes'].value_counts()

\N                      272319
22                       34419
30                       30559
43                       25519
23                       24765
                         ...  
308                          1
347                          1
356                          1
Game-Show,Reality-TV         1
Reality-TV                   1
Name: runtimeMinutes, Length: 335, dtype: int64

### Find Averages for Error Runtimes

In [48]:
# Fix cases with errors runtimes
errors_list = df_imdb_episodes.loc[df_imdb_episodes['runtimeMinutes'].str.isnumeric() == False, 'runtimeMinutes'].unique()
print('ERRORS:', errors_list)
df_runtime_errors = df_imdb_episodes.loc[df_imdb_episodes['runtimeMinutes'].isin(errors_list)]
print('# EPISODES WITH ERRORS:', len(df_runtime_errors))

# If series has episodes without Nan runtimes, use mean of the series (otherwise use mean of all series)
df_runtime_without_errors = df_imdb_episodes[~df_imdb_episodes['runtimeMinutes'].isin(errors_list)].copy()
df_runtime_without_errors['runtimeMinutes'] = df_runtime_without_errors['runtimeMinutes'].astype(int)

# Mean of all episode lengths
mean_runtime = df_runtime_without_errors['runtimeMinutes'].mean()
print('MEAN EPISODE LENGTH:', mean_runtime)

# Mean of episode length by series
df_mean_runtime_series = df_runtime_without_errors.groupby('parentTconst').agg({'runtimeMinutes': 'mean'})
df_mean_runtime_series.reset_index(inplace=True)

# 1. Use existing series info (use how='left' to keep series without runtimes for step 2)
df_runtime_errors = df_runtime_errors.merge(df_mean_runtime_series[['parentTconst', 'runtimeMinutes']], how='left', on='parentTconst')
df_runtime_errors.drop('runtimeMinutes_x', axis=1, inplace=True)
df_runtime_errors.rename(columns={'runtimeMinutes_y': 'runtimeMinutes'}, inplace=True)
print('# EPISODES WITH ERRORS AFTER USING MEAN PER SERIES:', len(df_runtime_errors[df_runtime_errors['runtimeMinutes'].isin(errors_list)]))

# 2. If there is no info for series, runtimeMinutes columns will have become Nans -> use global mean
print('# EPISODES WITHOUT RUNTIME DATA FROM SERIES TO FILL IN:', df_runtime_errors['runtimeMinutes'].isna().sum())
df_runtime_errors.loc[df_runtime_errors['runtimeMinutes'].isna(), 'runtimeMinutes'] = mean_runtime
print('# EPISODES WITH NANS AFTER USING GLOBAL MEAN OF SERIES:', df_runtime_errors['runtimeMinutes'].isna().sum())

ERRORS: ['\\N' 'Game-Show,Reality-TV' 'Game-Show,Reality-TV,Short' 'Reality-TV']
# EPISODES WITH ERRORS: 272322
MEAN EPISODE LENGTH: 38.68938414322491
# EPISODES WITH ERRORS AFTER USING MEAN PER SERIES: 0
# EPISODES WITHOUT RUNTIME DATA FROM SERIES TO FILL IN: 128079
# EPISODES WITH NANS AFTER USING GLOBAL MEAN OF SERIES: 0


### Define Runtime Metric

In [59]:
# Calculate total runtime per series
df_runtime_score = pd.concat([df_runtime_without_errors, df_runtime_errors])
print('# EPISODES AFTER REBUILDING EPISODE RUNTIMES:', len(df_runtime_score))   # Should match value_counts in the cells above
df_runtime_score = df_runtime_score.groupby('parentTconst').agg({'runtimeMinutes': 'sum'})
df_runtime_score.rename(columns={'runtimeMinutes': 'totalRuntime'}, inplace=True)
df_runtime_score['totalRuntime'] = round(df_runtime_score['totalRuntime']).astype(int)
df_runtime_score.reset_index(inplace=True)
df_runtime_score

# EPISODES AFTER REBUILDING EPISODE RUNTIMES: 696885


,parentTconst,totalRuntime
0,tt0039123,780
1,tt0039125,17
2,tt0040021,120
3,tt0040041,540
4,tt0040048,60
...,...,...
36653,tt9914546,50
36654,tt9914700,77
36655,tt9915144,232
36656,tt9916128,254


# Combined Metric

In [73]:
# Merge score with episode score
df_combined = pd.merge(
    left=df_series_score,
    right=df_episode_score,
    left_on='tconst',
    right_on='parentTconst'
)

df_combined

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,seriesScore,seriesRank,parentTconst,episodeScore,episodeRank
0,tt0903747,tvSeries,Breaking Bad,Breaking Bad,0,2008,2013,45,"Crime,Drama,Thriller",9.5,2100308,9.75,1,tt0903747,8.355000,5
1,tt0944947,tvSeries,Game of Thrones,Game of Thrones,0,2011,2019,4189,"Action,Adventure,Drama",9.2,2254188,9.60,2,tt0944947,8.638356,4
2,tt7366338,tvMiniSeries,Chernobyl,Chernobyl,0,2019,2019,330,"Drama,History,Thriller",9.3,848340,9.54,3,tt7366338,9.284000,1
3,tt1475582,tvSeries,Sherlock,Sherlock,0,2010,2017,88,"Crime,Drama,Mystery",9.1,986352,9.48,4,tt1475582,8.102667,7
4,tt0108778,tvSeries,Friends,Friends,0,1994,2004,5280,"Comedy,Romance",8.9,1074934,9.40,5,tt0108778,6.896068,3470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35958,tt6013216,tvMiniSeries,Obychnyi den shkolnika,Obychnyi den shkolnika,0,2016,\N,2,"Comedy,Thriller",1.0,29,3.00,109740,tt6013216,3.000000,36590
35959,tt2058840,tvSeries,Figgle Chat with Fred Figglehorn,Figgle Chat with Fred Figglehorn,0,2011,2011,\N,Comedy,1.0,130,3.00,109740,tt2058840,3.533333,36442
35960,tt11766306,tvSeries,Attaway General,Attaway General,0,2020,\N,11,Drama,1.0,2738,3.00,109740,tt11766306,3.725926,36370
35961,tt5016274,tvSeries,Biatches,Biatches,0,2014,2014,4,"Animation,Comedy",1.0,757,3.00,109740,tt5016274,3.208333,36508


In [74]:
# Merge with runtime score
df_combined = df_combined.merge(df_runtime_score, on='parentTconst')
df_combined.drop(columns='parentTconst', inplace=True)
df_combined

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,seriesScore,seriesRank,episodeScore,episodeRank,totalRuntime
0,tt0903747,tvSeries,Breaking Bad,Breaking Bad,0,2008,2013,45,"Crime,Drama,Thriller",9.5,2100308,9.75,1,8.355000,5,2941
1,tt0944947,tvSeries,Game of Thrones,Game of Thrones,0,2011,2019,4189,"Action,Adventure,Drama",9.2,2254188,9.60,2,8.638356,4,4214
2,tt7366338,tvMiniSeries,Chernobyl,Chernobyl,0,2019,2019,330,"Drama,History,Thriller",9.3,848340,9.54,3,9.284000,1,327
3,tt1475582,tvSeries,Sherlock,Sherlock,0,2010,2017,88,"Crime,Drama,Mystery",9.1,986352,9.48,4,8.102667,7,1211
4,tt0108778,tvSeries,Friends,Friends,0,1994,2004,5280,"Comedy,Romance",8.9,1074934,9.40,5,6.896068,3470,5287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35958,tt6013216,tvMiniSeries,Obychnyi den shkolnika,Obychnyi den shkolnika,0,2016,\N,2,"Comedy,Thriller",1.0,29,3.00,109740,3.000000,36590,39
35959,tt2058840,tvSeries,Figgle Chat with Fred Figglehorn,Figgle Chat with Fred Figglehorn,0,2011,2011,\N,Comedy,1.0,130,3.00,109740,3.533333,36442,232
35960,tt11766306,tvSeries,Attaway General,Attaway General,0,2020,\N,11,Drama,1.0,2738,3.00,109740,3.725926,36370,1045
35961,tt5016274,tvSeries,Biatches,Biatches,0,2014,2014,4,"Animation,Comedy",1.0,757,3.00,109740,3.208333,36508,232


In [77]:
df_combined = df_combined[
    [
        'tconst',
        'titleType',
        'primaryTitle',
        'originalTitle',
        'startYear',
        'endYear',
        'averageRating',
        'numVotes',
        'seriesScore',
        'seriesRank',
        'episodeScore',
        'episodeRank',
        'totalRuntime',
    ]
]

# Using numVotes to combine with runtime score
df_combined['combinedMetric'] = (df_combined['seriesScore'] * df_combined['episodeScore']) / 2 # Normal version
# df_combined = normalise_scores(df_combined, score_col='combinedMetric')
df_combined.sort_values('combinedMetric', ascending=False, inplace=True)
df_combined.head(15)

,tconst,titleType,primaryTitle,originalTitle,startYear,endYear,averageRating,numVotes,seriesScore,seriesRank,episodeScore,episodeRank,totalRuntime,combinedMetric
2,tt7366338,tvMiniSeries,Chernobyl,Chernobyl,2019,2019,9.3,848340,9.54,3,9.284000,1,327,44.284680
1,tt0944947,tvSeries,Game of Thrones,Game of Thrones,2011,2019,9.2,2254188,9.60,2,8.638356,4,4214,41.464110
0,tt0903747,tvSeries,Breaking Bad,Breaking Bad,2008,2013,9.5,2100308,9.75,1,8.355000,5,2941,40.730625
21,tt3581920,tvSeries,The Last of Us,The Last of Us,2023,\N,8.8,496636,8.86,22,9.063333,2,521,40.150567
3,tt1475582,tvSeries,Sherlock,Sherlock,2010,2017,9.1,986352,9.48,4,8.102667,7,1211,38.406640
13,tt2560140,tvSeries,Attack on Titan,Shingeki no Kyojin,2013,2023,9.1,494904,9.04,14,8.216939,6,2458,37.140563
5,tt4574334,tvSeries,Stranger Things,Stranger Things,2016,2025,8.7,1314465,9.33,6,7.852647,9,2070,36.632599
15,tt2085059,tvSeries,Black Mirror,Black Mirror,2011,\N,8.7,630421,9.02,15,7.990370,8,1599,36.036570
55,tt11198330,tvSeries,House of the Dragon,House of the Dragon,2022,\N,8.4,360418,8.26,54,8.706000,3,615,35.955780
9,tt3032476,tvSeries,Better Call Saul,Better Call Saul,2015,2022,9.0,632608,9.20,9,7.734603,13,3133,35.579175


# Finished and Unwatched

In [78]:
from datetime import datetime

df_watch = df_combined.loc[(df_combined['endYear'] != '\\N')]    # No nulls
df_watch = df_watch.loc[df_watch['endYear'].astype(int) <= datetime.now().year]    # No future endings

df_watch = df_watch.loc[~(df_watch['tconst'].isin(df_my_ratings['tconst']))]
df_watch.index = np.arange(1, 1+len(df_watch))
df_watch.to_csv('df_watch.csv')
df_watch.head(20)

,tconst,titleType,primaryTitle,originalTitle,startYear,endYear,averageRating,numVotes,seriesScore,seriesRank,episodeScore,episodeRank,totalRuntime,combinedMetric
1,tt2560140,tvSeries,Attack on Titan,Shingeki no Kyojin,2013,2023,9.1,494904,9.04,14,8.216939,6,2458,37.140563
2,tt0141842,tvSeries,The Sopranos,The Sopranos,1999,2007,9.2,459666,9.02,15,7.172209,1286,4733,32.346664
3,tt3322312,tvSeries,Daredevil,Daredevil,2015,2018,8.6,469855,8.68,34,7.402051,318,2080,32.124903
4,tt9140554,tvSeries,Loki,Loki,2021,2023,8.2,404538,8.26,54,7.746667,12,586,31.993733
5,tt4158110,tvSeries,Mr. Robot,Mr. Robot,2015,2019,8.5,414074,8.48,46,7.459556,223,2199,31.628516
6,tt0475784,tvSeries,Westworld,Westworld,2016,2022,8.5,528568,8.74,30,7.233333,975,2185,31.609667
7,tt1520211,tvSeries,The Walking Dead,The Walking Dead,2010,2022,8.1,1068143,8.97,18,6.990339,2534,8105,31.351670
8,tt0306414,tvSeries,The Wire,The Wire,2002,2008,9.3,371496,8.86,22,7.017667,2223,3566,31.088263
9,tt0417299,tvSeries,Avatar: The Last Airbender,Avatar: The Last Airbender,2005,2008,9.3,355085,8.80,26,7.048226,2090,1504,31.012194
10,tt2802850,tvSeries,Fargo,Fargo,2014,2024,8.9,413212,8.72,31,6.973137,2644,2647,30.402878
